# VectorStore-Backed Memory

`VectorStoreRetrieverMemory` stores interactions in a VectorDB and queries the top-K most "salient" interactions every type it is called.

This differs from most of the other Memory classes in that 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain

### Initialize your VectorStore

Depending on the store you choose, this step may look different. Consult the relevant VectorStore documentation for more details.

In [2]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

### Create your the VectorStoreRetrieverMemory

In [3]:
memory = VectorStoreRetrieverMemory(
    retriever=vectorstore.as_retriever(),
    return_docs=False, # Makes load_memory_variables() return Dict[str, str] instead of Dict[str, List[Document]]
    )

# Examples of how the memory object is called
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "Heard of France?"}, {"output": "Undoubtedly"})
memory.save_context({"input": "How about Australia?"}, {"output": "Most certainly"})
memory.save_context({"input": "What about Montreal?"}, {"output": "Of course."})

In [4]:
# Notice the first result discusses France, which the language model deems more semantically relevant
# than the other locations
print(memory.load_memory_variables({"prompt": "Emmanuel Macron"})["history"])

input: Heard of France?
output: Undoubtedly
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
input: hi
output: whats up


## Using in a chain
Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [5]:
llm = OpenAI() # Can be any valid LLM
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: hi
output: whats up
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
input: Heard of France?
output: Undoubtedly
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great, just thinking about all of the wonderful places in the world. How about you?"

In [6]:
conversation_with_summary.predict(input="What do you know about Uluru?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Hi, what's up?
response:  Hi there! I'm doing great, just thinking about all of the wonderful places in the world. How about you?
input: How about Australia?
output: Most certainly
input: Heard of France?
output: Undoubtedly
input: hi
output: whats up
Human: What do you know about Uluru?
AI:

> Finished chain.


' Uluru is the aboriginal name for Ayers Rock, a large sandstone rock formation in the Northern Territory of Australia. It is considered a sacred site for the indigenous people of Australia.'

In [7]:
conversation_with_summary.predict(input="Tell me more about UNESCO!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Heard of France?
output: Undoubtedly
input: What about Montreal?
output: Of course.
input: Hi, what's up?
response:  Hi there! I'm doing great, just thinking about all of the wonderful places in the world. How about you?
input: What do you know about Uluru?
response:  Uluru is the aboriginal name for Ayers Rock, a large sandstone rock formation in the Northern Territory of Australia. It is considered a sacred site for the indigenous people of Australia.
Human: Tell me more about UNESCO!
AI:

> Finished chain.


' Sure! UNESCO is the United Nations Educational, Scientific and Cultural Organization. It was founded in 1945 and works to promote peace and security through international collaboration in the realms of education, science, culture and communication. UNESCO is based in Paris, France and is responsible for the protection of over 1000 World Heritage Sites around the world.'

In [8]:
# We can see here that the buffer is updated
conversation_with_summary.predict(input="Are there UNESCO sites in Canada?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Tell me more about UNESCO!
response:  Sure! UNESCO is the United Nations Educational, Scientific and Cultural Organization. It was founded in 1945 and works to promote peace and security through international collaboration in the realms of education, science, culture and communication. UNESCO is based in Paris, France and is responsible for the protection of over 1000 World Heritage Sites around the world.
input: What about Montreal?
output: Of course.
input: Heard of France?
output: Undoubtedly
input: How about Australia?
output: Most certainly
Human: Are there UNESCO sites in Canada?
AI:

> Finished chain.


' Yes, there are 17 UNESCO World Heritage Sites in Canada. They include the Rideau Canal, Old Town Lunenburg, the Landscape of Grand Pré, and the Historic District of Old Québec, among others.'